<a href="https://colab.research.google.com/github/bacchus00/aportantes-cmf/blob/main/rdfing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 4.4 MB/s 
     |████████████████████████████████| 41 kB 530 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
import urllib.parse

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Bushe/Trabajo Dirigido/cleaner_data.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9,10,14,17,18,22,25,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
pd.set_option("display.max_rows", None)
print(df['Descripcion1'].unique())

['Reembolso solicitado candidato o partido' 'No definido'
 'Aporte reservado' 'Aporte anónimo' 'Aporte patrimonial' 'Aporte público'
 'Anticipo financiamiento fiscal'
 'Reembolso facturas cedidas partido politico' 'Aporte propio' nan
 'Crédito institución financiera'
 'Intereses crédito institución financiera'
 'Aporte público partido politico' 'Aporte trabajo voluntario'
 'Reembolsos facturas endosadas' 'Aporte por Usufructo'
 'Aportes en dinero recibido por candidato de un partido politico'
 'Aportes Reservado' 'Aporte personal del candidato' 'Aportes Anónimos'
 'Intereses devengados por créditos con Instituciones Financieras'
 'Aporte por trabajo voluntario' 'Créditos con Instituciones Financieras'
 'Reembolso solicitado por cesión de derecho a partido político'
 'Aportes Privados de Carácter Público' 'Usufructo'
 'Reembolsos solicitado por endosos' 'Aporte Propio'
 'Aportes Privados de Carácter Público de Partidos Politicos'
 'Intereses por crédito con Institución Financiera' 'APOR

In [ ]:
# list of ingresos that actually are contributions
ingresos_def = [
  'No definido'
  'Aporte reservado'
  'Aporte anónimo'
  'Aporte patrimonial'
  'Aporte público'
  'Aporte propio'
  'Aporte público partido politico',
  'Aporte trabajo voluntario',
  'Aporte por Usufructo',
  'Aportes en dinero recibido por candidato de un partido politico',
  'Aportes Reservado',
  'Aporte personal del candidato',
  'Aportes Anónimos',
  'Aporte por trabajo voluntario',
  'Aportes Privados de Carácter Público',
  'Usufructo',
  'Aporte Propio',
  'Aportes Privados de Carácter Público de Partidos Politicos',
  'APORTE PROPIO',
  'Aportes de personas naturales',
  'Aportes en especies recibido por candidato de un partido politico',
  'Aporte estimado por uso de un vehículo de un tercero',
  'Aporte estimado por propaganda en espacio privado',
  'Aporte estimado por servicios profesionales de un tercero',
  'Aportes menores sin publicidad',
  'Aporte estimado en especies (mantención y colaciones)',
  'Aporte personal del candidato (vehículo, propiedades entre otros)',
  'Aporte estimado por uso de propiedad de un tercero' 'Comodato',
  'Aporte estimado por propaganda en medios de comunicación digital',
  'Aporte estimado por propaganda en radioemisoras',
  'Aporte estimado por propaganda escrita',
  'Donaciones',
  'Mutuos',
  'Otros actos o contratos a título gratuito',
  'Aportes propios de partido politico',
  'Aportes en especies o servicios de terceros',
  'Aporte con  Publicidad',
  'Aportes en dinero de Partidos Políticos',
  'Aporte propio en especies o servicios',
  'Aportes en especies o servicios de partidos politicos',
  'Aporte propio en efectivo del candidato',
  'Aportes sin Publicidad',
  'Financiamiento a partidos politicos',
  'Aportes con publicidad',
  'Aportes sin publicidad',
  'Aporte Propio por Imputacion de reembolso a candidatos',
  'Aportes Partido Politico',
  'Aporte Propio por Imputacion de reembolso a instituciones financieras',
  'Financiamiento a candidatos independientes',
  'Cesion de derecho a reembolso',
  'Imputación de gasto electoral de primarias',
  'Imputación saldo de financiamiento primarias',
  'Imputación de gasto presunto de partidos políticos',
  'Aportes de Partido politico',
  'Aportes con Publicidad',
  'Aportes Partidos politicos']

In [ ]:
# filtering only contributions
aportes = df[df['Descripcion1'].isin(ingresos_def)].sample(50)
aportes

,Archivo,Fecha,Candidato,RUTCandidato,Aportante,RUTAportante,Monto,Eleccion,TerritorioElectoral,Comuna,...,Afiliacion,TipoDocumento,GlosaDocumento,NumeroDocumento,TipoCuenta,ParaReembolso,EstadoCuenta,Descripcion1,Descripcion2,TipoPlanilla
435998,2021-presidente-senadores-diputados-CORE-ingre...,2021-10-27,MARIA VICTORIA RODRIGUEZ HERRERA,7300237-0,Formulario 104,NaN,30000.0,CONSEJERO REGIONAL,LOS ANDES,NaN,...,INDPENDIENTE DE PACTO,ND,Autorizacion de Propaganda en Espacios Privados,227957.0,121.0,NaN,NaN,Aportes en especies o servicios de terceros,No definido,INGRESOS
221165,2016-alcalde-consejal-ingresos-candidatos.xlsx,2016-10-11,JAVIER RAMIREZ GONZALEZ,13832744-2,SARA ZUÑIGA,f9d604e5411e654bd7f5532175c2bdca,12000.0,CONCEJAL,NaN,MELIPILLA,...,Afiliado,OI,VALORIZACION ESPACIOS PRIVADOS,17,146.0,NaN,NaN,Aporte estimado por propaganda en espacio privado,Otros Documentos de Ingresos,NaN
98417,2012-alcalde-consejal-ingresos-candidatos.xlsx,2012-09-24,RAUL LEIVA,12636469-5,-,NaN,68400.0,ALCALDE,NaN,TALAGANTE,...,Independiente en Pacto,CB,APORTE RESERVADO,1.0,130.0,NaN,NaN,Aportes Reservado,Cartola bancaria,I
129905,2016-alcalde-consejal-ingresos-candidatos.xlsx,2016-10-18,MARIA PAOLA OLGUIN GODOY,13855130-K,APORTE PERSONAL DELCANDIDATO,13855130-K,250000.0,CONCEJAL,NaN,QUILPUE,...,Afiliado,CB,APORTE PROPIO,1412431.0,110.0,NaN,NaN,Aporte personal del candidato,Cartola bancaria,NaN
296252,2021-alcaldes-consejales-CC-GORE-ingresos-cand...,2021-06-22,ROBERTO ANTONIO VILLAGRA REYES,8302925-0,Formulario 103,NaN,1040000.0,ALCALDE,RANCAGUA,NaN,...,INDPENDIENTE DE PACTO,ND,Registro de Vehículos,130186,121.0,NaN,NaN,Aportes en especies o servicios de terceros,No definido,INGRESOS
203993,2016-alcalde-consejal-ingresos-candidatos.xlsx,2016-10-04,RAUL VALDES ARIAS,11361368-8,VIVIANA ROBLESVASQUEZ,0b1c5da4785c3b1613baa3cfbbb56613,186158.0,CONCEJAL,NaN,CONCHALI,...,Independiente en SubPacto,OI,2 INSTALACION PALOMA EN CASA,2,146.0,NaN,NaN,Aporte estimado por propaganda en espacio privado,Otros Documentos de Ingresos,NaN
358726,2021-alcaldes-consejales-CC-GORE-ingresos-cand...,2021-05-24,ROBERTO CARDENAS SILVA,13527051-2,Formulario 101,NaN,80000.0,ALCALDE,TORRES DEL PAINE,NaN,...,INDPENDIENTE DE PACTO,ND,Registro de Brigadistas y Voluntarios,81965,121.0,NaN,NaN,Aportes en especies o servicios de terceros,No definido,INGRESOS
314409,2021-alcaldes-consejales-CC-GORE-ingresos-cand...,2021-04-06,VERONICA DEL CARMEN ARIAS PEDRAZA,9961141-3,Formulario 104,NaN,45000.0,CONCEJAL,EL QUISCO,NaN,...,INDPENDIENTE DE PACTO,ND,Autorización de Propaganda en Espacios Privados,47262,121.0,NaN,NaN,Aportes en especies o servicios de terceros,No definido,INGRESOS
94825,2012-alcalde-consejal-ingresos-candidatos.xlsx,2012-08-10,ORLANDO MORALES,10839813-2,-,NaN,15000.0,CONCEJAL,NaN,LA CISTERNA,...,Afiliado,OI,transferencia electronica,84355164.0,120.0,NaN,NaN,Aportes Anónimos,Otros Documentos de Ingresos,I
107104,2013-presidente-senador-diputado-CORE-ingresos...,2013-09-01,PATRICIO BASSO,4487229-3,LEONARDO JAVIER BASS O SOTZ,e62256d99750c61407928fec51e0010c,1000000.0,DIPUTADO,NaN,NaN,...,Afiliado,CB,TRASPASO DESDE OTRO BANCO,0.0,140.0,NaN,Aprobada,Aportes Privados de Carácter Público,Cartola bancaria,I


In [ ]:
# namespaces
ctbr = Namespace("http://example.org/contributor/")
ctb = Namespace("http://example.org/contribution/")
prty = Namespace("http://example.org/party/")
elec = Namespace("http://example.org/election/")
cnd = Namespace("http://example.org/candidate/")
trry = Namespace("http://example.org/territory/")

#custom properties
rut = URIRef(Namespace("http://example.org/my-schema/rut/"))
contributed = URIRef(Namespace("http://example.org/my-schema/contributed/"))
recievedBy = URIRef(Namespace("http://example.org/my-schema/recievedBy/"))
forElection = URIRef(Namespace("http://example.org/my-schema/forElection/"))
year = URIRef(Namespace("http://example.org/my-schema/year/"))

#predefined properties
schema = Namespace('http://schema.org/')

In [ ]:
# function to transform strings to form valid iris
def iri_str(x):
  a_string = x.replace(' ','-')
  a_string = a_string.replace('Ñ','N')
  return a_string

In [ ]:
g = Graph()

for index, row in aportes.iterrows():
  #check for nans
  has_contributor = row['Aportante']==row['Aportante']
  has_rut_contributor = row['RUTAportante']==row['RUTAportante']
  has_party = row['Partido']==row['Partido']
  has_rut_party = row['RUTPartido']==row['RUTPartido']
  has_candidate = row['Candidato']==row['Candidato']
  has_rut_candidate = row['RUTCandidato']==row['RUTCandidato']
  has_date = row['Fecha']==row['Fecha']

  #add contributor
  giver = ''
  if has_contributor:
    contributor = URIRef(ctbr+iri_str(row['Aportante']))
    giver = row['Aportante']
    #g.add((contributor, RDF.type, FOAF.Person))
    g.add((contributor, URIRef(schema+'name'), Literal(row['Aportante'], datatype=XSD.string)))
    if has_rut_contributor:
      g.add((contributor, rut, Literal(row['RUTAportante'], datatype=XSD.string)))

  #add party
  if has_party:
    party = URIRef(prty+iri_str(row['Partido']))
    reciever = row['Partido']
    reciever_iri = party
    g.add((party, URIRef(schema+'name'), Literal(row['Partido'], datatype=XSD.string)))
    if has_rut_party:
      g.add((party, rut, Literal(row['RUTPartido'], datatype=XSD.string)))

  #add candidate
  if has_candidate:
    candidate = URIRef(cnd+iri_str(row['Candidato']))
    reciever = row['Candidato']
    reciever_iri = candidate
    g.add((candidate, RDF.type, FOAF.Person))
    g.add((candidate, URIRef(schema+'name'), Literal(row['Candidato'], datatype=XSD.string)))
    if has_rut_candidate:
      g.add((candidate, rut, Literal(row['RUTCandidato'], datatype=XSD.string)))
    if has_party:
      g.add((candidate, URIRef(schema+'memberOf'), party))


  #add election
  election = URIRef(elec+iri_str(row['Eleccion']))
  g.add((election, year, Literal(row['Archivo'].split('-')[0], datatype=XSD.integer)))
  territories = [
    'TerritorioElectoral', 
    'Comuna',
    'Region', 
    'Distrito',
    'Circunscripcion', 
    'CircunscripcionSenatorial',
    'CircunscripcionProvincial']
  for territory in territories:
    if row[territory] == row[territory]:
      g.add((election, URIRef(schema+'location'), Literal(row[territory], datatype=XSD.string)))
    
  #add contribution
  contribution = URIRef(ctb + iri_str(giver) +'-a-' + iri_str(reciever) + '-' + str(row['Fecha'])+'-'+str(index))
  g.add((contribution, URIRef(schema+'amount'), Literal(int(row['Monto']), datatype=XSD.int)))
  if has_date:
    g.add((contribution, URIRef(schema+'datePosted'), Literal(row['Fecha'], datatype=XSD.date)))
  if has_contributor:
    g.add((contributor, contributed, contribution))
  g.add((contribution, recievedBy, reciever_iri))
  g.add((contribution, forElection, election))

In [ ]:
# resulting graph
print(g.serialize(format='turtle'))

@prefix ns1: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/contribution/-a-PABLO-KAST-SOMMERHOFF-2017-11-08-242303> <http://example.org/my-schema/forElection/> <http://example.org/election/DIPUTADO> ;
    <http://example.org/my-schema/recievedBy/> <http://example.org/candidate/PABLO-KAST-SOMMERHOFF> ;
    ns1:amount "300000"^^xsd:int ;
    ns1:datePosted "2017-11-08"^^xsd:date .

<http://example.org/contribution/-a-WASHINGTON-SHLEGER-DERBY-JERIA-2016-11-08-200447> <http://example.org/my-schema/forElection/> <http://example.org/election/CONCEJAL> ;
    <http://example.org/my-schema/recievedBy/> <http://example.org/candidate/WASHINGTON-SHLEGER-DERBY-JERIA> ;
    ns1:amount "548605"^^xsd:int ;
    ns1:datePosted "2016-11-08"^^xsd:date .

<http://example.org/contributor/-> <http://example.org/my-schema/contributed/> <http://example.org/contribution/--a-ORLANDO-MORALES-2012-08-10-94825>,
        <http://example.org/contribution/--a-RAUL-LEIVA

In [ ]:
# saving results to ttl
g.serialize(destination="/content/drive/MyDrive/Bushe/Trabajo Dirigido/aportes_sample.ttl")

<Graph identifier=Ne4705d1e08d140488fcdac96587ac78a (<class 'rdflib.graph.Graph'>)>